In [1]:
!pip install ta
!pip install arch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for oandapyV20: filename=oandapyV20-0.7.2-py3-none-any.whl size=69778 sha256=2d1973071d2c2dae2e9c1832d9ab1fe429c838b7aa8670e4227f21f1067b9533
  Stored in directory: /root/.cache/pip/wheels/d7/4f/5c/9bfd8c96f1a36dccb743e81d8718ccde66454d9cda2427262e
Successfully built oandapyV20
  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=37814c18eeea7d03c6d001101a238e09d6f5d7a182273e172f12156e7f944187
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.3/985.3 kB 11.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, train_test_split
from tqdm import tqdm
from arch import arch_model
import lightgbm as lgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam



/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
#Feature Engineering
# Load data
df = pd.read_csv('forex_data.csv', index_col='time', parse_dates=True)

# Create time features
df['day_of_week'] = df.index.dayofweek
df['month'] = df.index.month
df['day_of_month'] = df.index.day
df['week_of_year'] = df.index.isocalendar().week
import ta
# Calculate technical indicators
# RSI
df['rsi'] = ta.momentum.RSIIndicator(df['close']).rsi()

# MACD
macd = ta.trend.MACD(df['close'])
df['macd'] = macd.macd()
df['macd_signal'] = macd.macd_signal()
df['macd_hist'] = macd.macd_diff()

# Bollinger Bands
bb = ta.volatility.BollingerBands(df['close'])
df['bb_high'] = bb.bollinger_hband()
df['bb_low'] = bb.bollinger_lband()
df['bb_mavg'] = bb.bollinger_mavg()
df['bb_width'] = bb.bollinger_wband()

# Function to create lagged features
def create_lag_features(df, lags):
    for lag in range(1, lags + 1):
        df[f'lag_{lag}'] = df['close'].shift(lag)
    return df

df = create_lag_features(df, lags=31)

# Function to create rolling mean and standard deviation features
def create_rolling_features(df, windows):
    for window in windows:
        df[f'rolling_mean_{window}'] = df['close'].rolling(window=window).mean()
        df[f'rolling_std_{window}'] = df['close'].rolling(window=window).std()
    return df

df = create_rolling_features(df, windows=[3, 7, 14, 30, 50])

# Define the target variable as the next day's close price
df['target'] = df['close'].shift(-1)

# Fit a GARCH model on the residuals
returns = df['close'].pct_change().dropna()  # Use percentage change as returns
garch_model = arch_model(returns, vol='Garch', p=1, q=1)
garch_fit = garch_model.fit(disp='off')

# Get the conditional volatility from the GARCH model
df['garch_vol'] = garch_fit.conditional_volatility

# Drop NaN values created by GARCH model
df.dropna(inplace=True)
df.to_csv('Forex_data_feature_added (Garch).csv')

# Load the data
df=pd.read_csv('Forex_data_feature_added (Garch).csv', index_col='time', parse_dates=True)



/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 2.463e-05. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
